# Goals

1. visualize DRB with folium
2. determine how to cut it into 2 degree chips



In [1]:
import boto3

In [4]:
! aws s3 ls dev-et-data/out_drb_20years/dd_1999.tif

2020-06-23 21:51:47   48410542 dd_1999.tif


# rio takes too long - research ways to just read the header

## USGS network security should be investigated again for slow speed ???

In [5]:
! rio info /vsis3/dev-et-data/out_drb_20years/dd_1999.tif

{"bounds": [-77.01862410699997, 37.02493923991505, -72.97939437249997, 43.50934926191505], "colorinterp": ["gray"], "count": 1, "crs": "EPSG:4326", "descriptions": [null], "driver": "GTiff", "dtype": "float64", "height": 3116, "indexes": [1], "interleave": "band", "lnglat": [-74.99900923974997, 40.26714425091505], "mask_flags": [["all_valid"]], "nodata": null, "res": [0.0020810045, 0.0020810045], "shape": [3116, 1941], "tiled": false, "transform": [0.0020810045, 0.0, -77.01862410699997, 0.0, -0.0020810045, 43.50934926191505, 0.0, 0.0, 1.0], "units": [null], "width": 1941}


In [6]:
import fsspec
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)
chip_list = fs.ls('dev-et-data/tiles/')

In [7]:
import rasterio
file='/vsis3/dev-et-data/out_drb_20years/dd_1999.tif'
src = rasterio.open(file)
src.shape

(3116, 1941)

In [8]:
#dir(src)

src.bounds.left

-77.01862410699997

In [10]:
#dir(src.bounds)



You are almost there with what you tried, and using the box method is indeed the best way. With the list comprehension, you can do

b = [box(l, b, r, t) for l, b, r, t in zip(df.left, df.bottom, df.right, df.top)]

Another option is to apply the box function to each row of your dataframe:

b = df.apply(lambda row: box(row.left, row.bottom, row.right, row.top), axis=1)

Once you have converted the bounding boxes to polygons, make sure to actually create a GeoDataFrame:

gdf = geopandas.GeoDataFrame(df, geometry=b)



In [11]:
# chip_list

In [11]:
def get_bounding(file):
    try:
        src = rasterio.open(file)
        return(src.bounds)
    except:
        pass
        

In [17]:
def poly_from_bounds(bounds):
    coord_list = []
    ul_lon_lat=[bounds.left,bounds.top]
    ur_lon_lat=[bounds.right, bounds.top]
    lr_lon_lat=[bounds.right,bounds.bottom]
    ll_lon_lat=[bounds.left,bounds.bottom]
    print (ul_lon_lat)
    coord_list.append(ul_lon_lat)
    print (ur_lon_lat)
    coord_list.append(ur_lon_lat)
    print (lr_lon_lat)
    coord_list.append(lr_lon_lat)
    print (ll_lon_lat)
    coord_list.append(ll_lon_lat)
    print (ul_lon_lat)
    coord_list.append(ul_lon_lat)
    return(coord_list)
    pass

In [18]:
filename='/vsis3/dev-et-data/out_drb_20years/dd_1999.tif'

bounds = get_bounding(filename)
bounds

BoundingBox(left=-77.01862410699997, bottom=37.02493923991505, right=-72.97939437249997, top=43.50934926191505)

In [19]:
poly = poly_from_bounds(bounds)

[-77.01862410699997, 43.50934926191505]
[-72.97939437249997, 43.50934926191505]
[-72.97939437249997, 37.02493923991505]
[-77.01862410699997, 37.02493923991505]
[-77.01862410699997, 43.50934926191505]


In [21]:
def build_geojson(coord_list):
    print(coord_list)
    polycs = []
    polycs.append(coord_list)
    geos = []
    for polyc in polycs:
        poly = {
            'type': 'Feature',
            'properties': {},
            'geometry': {
                'type': 'Polygon',
                'coordinates': [polyc]
            }
        }
        geos.append(poly)

    geometries = {
       'type': 'FeatureCollection',
        'features': geos,
    }
    return geometries

In [23]:
geo_json_str = build_geojson(poly)
geo_json_str

[[-77.01862410699997, 43.50934926191505], [-72.97939437249997, 43.50934926191505], [-72.97939437249997, 37.02493923991505], [-77.01862410699997, 37.02493923991505], [-77.01862410699997, 43.50934926191505]]


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-77.01862410699997, 43.50934926191505],
      [-72.97939437249997, 43.50934926191505],
      [-72.97939437249997, 37.02493923991505],
      [-77.01862410699997, 37.02493923991505],
      [-77.01862410699997, 43.50934926191505]]]}}]}

In [26]:
import folium


In [27]:
tony_plot_tiles_json = geo_json_str

In [28]:

map_osm = folium.Map(location=[35, -90], zoom_start=5)

tile_boxes=folium.features.GeoJson(tony_plot_tiles_json)
map_osm.add_child(tile_boxes)
map_osm

In [38]:
def round_box(bounds):
    box={}
    box={'left': bounds.left, 'bottom': bounds.bottom , 'right': bounds.right, 'top': bounds.top}
    print(bounds)
    box={'left': -78, 'bottom':36 , 'right': -72, 'top': 44}
    return box

In [39]:
box=round_box(bounds)
box

BoundingBox(left=-77.01862410699997, bottom=37.02493923991505, right=-72.97939437249997, top=43.50934926191505)


{'left': -78, 'bottom': 36, 'right': -72, 'top': 44}

In [40]:
def poly_from_box(box):
    coord_list = []
    ul_lon_lat=[box['left'], box['top']]
    ur_lon_lat=[box['right'], box['top']]
    lr_lon_lat=[box['right'], box['bottom']]
    ll_lon_lat=[box['left'], box['bottom']]
    print (ul_lon_lat)
    coord_list.append(ul_lon_lat)
    print (ur_lon_lat)
    coord_list.append(ur_lon_lat)
    print (lr_lon_lat)
    coord_list.append(lr_lon_lat)
    print (ll_lon_lat)
    coord_list.append(ll_lon_lat)
    print (ul_lon_lat)
    coord_list.append(ul_lon_lat)
    return(coord_list)

In [42]:
poly2=poly_from_box(box)

[-78, 44]
[-72, 44]
[-72, 36]
[-78, 36]
[-78, 44]


In [43]:
geo_json_str2 = build_geojson(poly2)


[[-78, 44], [-72, 44], [-72, 36], [-78, 36], [-78, 44]]


In [44]:
tile_boxes=folium.features.GeoJson(geo_json_str2)
map_osm.add_child(tile_boxes)
map_osm

In [47]:
def chip_list(box):
        CHIP_LIST = []
        

        starting_lat = box['top']
        ending_lat = box['bottom']

        print(starting_lat, ending_lat)

        starting_lon = box['left']
        ending_lon = box['right']
        print(starting_lon, ending_lon)

        lat = starting_lat
        while lat > ending_lat:
            print(lat)

            lon = starting_lon
            while lon < ending_lon:
                print(lon)
                chip_name = 'chip' + str(lat) + 'N' + str(lon) +'E'
                CHIP_LIST.append(chip_name)
                lon = lon + 2

            lat = lat - 2

        return CHIP_LIST

In [48]:
cl = chip_list(box)

44 36
-78 -72
44
-78
-76
-74
42
-78
-76
-74
40
-78
-76
-74
38
-78
-76
-74


In [49]:
cl

['chip44N-78E',
 'chip44N-76E',
 'chip44N-74E',
 'chip42N-78E',
 'chip42N-76E',
 'chip42N-74E',
 'chip40N-78E',
 'chip40N-76E',
 'chip40N-74E',
 'chip38N-78E',
 'chip38N-76E',
 'chip38N-74E']

In [50]:
len(cl)

12